# Recipe Recommendation System 
***THE BOMA POT***

### **Author:** ***DS Martin Waweru***

## Table Of Contents

- Business Understanding
- Data Understanding
- Data Preparation
- Modelling 
- Evaluation 
  

## Business Understanding

### Business Description 
The Boma Pot is a web-based culinary platform dedicated to sharing authentic African recipes with the world. Our mission is to enhance the cooking experience of home chefs by providing them with a diverse collection of recipes rooted in African culinary traditions, while also incorporating global influences. Whether you're an experienced cook or just starting, Tasty Dishes offers a wide variety of recipes that empower users to create delicious, flavorful meals from the comfort of their homes.


## Business Goal 
### Objective
The main objective of this project is to develop an item-based recipe recommendation system that suggests recipes to users based on the ingredients they have available. By analyzing the ingredients present in various recipes, the system aims to provide relevant and appealing recommendations that encourage users to explore and cook diverse dishes rooted in African culinary traditions, while also incorporating global flavors.

### Scope

1. Ingredient-Based Recommendations: Develop an algorithm that analyzes user-provided ingredients to recommend recipes based on ingredient similarity, leveraging a diverse dataset that includes recipe_Title, Ingredients, and Instructions for authentic African and global dishes.

2. User-Friendly Interface: Design an intuitive web interface that enables users to input their available ingredients and view tailored recipe recommendations, along with detailed cooking instructions and a feedback mechanism to enhance recommendation accuracy.

## Data Understanding

### Data Source:

We are working with two datasets. The first is from the Food and Agriculture Organization of the United Nations, in collaboration with Kenya's Ministry of Agriculture and Livestock Development. Originally in PDF format, the necessary information was extracted and converted into a CSV file. The second dataset comes from Kaggle, titled the Food Ingredients and Recipe Dataset with Image Name Mapping, which provides global recipe data.

### Access data
click this link below:

1. [Ministry of Agriculture](https://kilimo.go.ke/wp-content/uploads/2021/01/KENYA-FOOD-RECIPES-2018.pdf)
2. [Food Ingredients and Recipe Dataset](https://www.kaggle.com/datasets/pes12017000148/food-ingredients-and-recipe-dataset-with-images)

In [1]:
# Necessary Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import string
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


## Data Preparation

### Data Frame One
The dataset contains a CSV file and a zipped folder, consisting of 13,501 rows  and 5 columns respectively as follows:
1. Title: Represents the Title of the Food Dish.
2. Ingredients: Contains the ingredients as they were scraped from the website.
3. Instructions: Has the recipe instructions to be followed to recreate the dish.
4. Image_Name: Has the name of the image as stored in the Food Images zipped folder.
5. Cleaned_Ingredients: Contains the ingredients after being processed and cleaned.

In [2]:
# Load the dataframe

df = pd.read_csv("Data\Food Ingredients and Recipe Dataset with Image Name Mapping.csv", index_col=0)


# Display the first columns
display(df.head(10))

#show the shape
print(df.shape)

,Title,Ingredients,Instructions,Image_Name,Cleaned_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...",miso-butter-roast-chicken-acorn-squash-panzanella,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher..."
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,crispy-salt-and-pepper-potatoes-dan-kluger,"['2 large egg whites', '1 pound new potatoes (..."
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...,thanksgiving-mac-and-cheese-erick-williams,"['1 cup evaporated milk', '1 cup whole milk', ..."
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...,italian-sausage-and-bread-stuffing-240559,"['1 (¾- to 1-pound) round Italian loaf, cut in..."
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...,newtons-law-apple-bourbon-cocktail,"['1 teaspoon dark brown sugar', '1 teaspoon ho..."
5,Warm Comfort,"['2 chamomile tea bags', '1½ oz. reposado tequ...",Place 2 chamomile tea bags in a heatsafe vesse...,warm-comfort-tequila-chamomile-toddy,"['2 chamomile tea bags', '1½ oz. reposado tequ..."
6,Apples and Oranges,"['3 oz. Grand Marnier', '1 oz. Amaro Averna', ...","Add 3 oz. Grand Marnier, 1 oz. Amaro Averna, a...",apples-and-oranges-spiked-cider,"['3 oz. Grand Marnier', '1 oz. Amaro Averna', ..."
7,Turmeric Hot Toddy,"['¼ cup granulated sugar', '¾ tsp. ground turm...","For the turmeric syrup, combine ½ cup hot wate...",turmeric-hot-toddy-claire-sprouse,"['¼ cup granulated sugar', '¾ tsp. ground turm..."
8,Instant Pot Lamb Haleem,"['¾ cup assorted dals (such as chana dal, moon...","Combine dals, rice, and barley in a medium bow...",instant-pot-lamb-haleem,"['¾ cup assorted dals (such as chana dal, moon..."
9,Spiced Lentil and Caramelized Onion Baked Eggs,"['1 (14.5-ounce) can basic lentil soup, like A...","Place an oven rack in the center of the oven, ...",spiced-lentil-and-caramelized-onion-baked-eggs,"['1 (14.5-ounce) can basic lentil soup, like A..."


(13501, 5)


### Data Cleaning and Sanity check up

1. Check for duplicates and dropping them
2. checking for missing values, drop them or fill them
3. dropping ingredents column and renaming and rearrange the cleaned_ingredients to ingredients

- Check for duplicates

In [3]:
# checking shape of our data
df.shape

(13501, 5)

In [4]:
print(f'Number of duplicates: {df.duplicated().sum()}')

Number of duplicates: 0


In [5]:
#drop duplicated rows
df.drop_duplicates(inplace=True)
print(f'Number of duplicates after dropping: {df.duplicated().sum()}')

Number of duplicates after dropping: 0


- Check for missing values

In [6]:
df.isnull().sum().sort_values(ascending=False)

Instructions           8
Title                  5
Ingredients            0
Image_Name             0
Cleaned_Ingredients    0
dtype: int64

In [7]:
#drop rows with missing values
df.dropna(inplace=True)
print(f'Number of missing values after dropping: {df.isnull().sum().sum()}')

Number of missing values after dropping: 0


- There is the Ingredients and Cleaned Ingredients column, check if there is any difference between the two.

In [8]:
df['Ingredients'][5]

"['2 chamomile tea bags', '1½ oz. reposado tequila', '¾ oz. fresh lemon juice', '1 Tbsp. agave nectar']"

In [9]:
df['Cleaned_Ingredients'][5]

"['2 chamomile tea bags', '1½ oz. reposado tequila', '¾ oz. fresh lemon juice', '1 Tbsp. agave nectar']"

- There's no significant difference between Ingredients and cleaned Ingredients. Thus, we drop the Ingredients column and rename cleaned_ingredients  ingredients.

In [10]:
#move the cleaned_ingredients column to the second column
df = df[['Title', 'Cleaned_Ingredients', 'Ingredients', 'Instructions', 'Image_Name']]

#drop the ingredients column
df = df.drop(columns=['Ingredients','Image_Name'])


In [11]:
# rename the cleaned ingredients column
df = df.rename(columns={'Cleaned_Ingredients':'Ingredients'})
df.head()


,Title,Ingredients,Instructions
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ..."
1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...
2,Thanksgiving Mac and Cheese,"['1 cup evaporated milk', '1 cup whole milk', ...",Place a rack in middle of oven; preheat to 400...
3,Italian Sausage and Bread Stuffing,"['1 (¾- to 1-pound) round Italian loaf, cut in...",Preheat oven to 350°F with rack in middle. Gen...
4,Newton's Law,"['1 teaspoon dark brown sugar', '1 teaspoon ho...",Stir together brown sugar and hot water in a c...


In [12]:
df['Ingredients'][5]

"['2 chamomile tea bags', '1½ oz. reposado tequila', '¾ oz. fresh lemon juice', '1 Tbsp. agave nectar']"

### DataFrame two
The dataset is CSV file, it contains rows 142 and 16 columns respectively as follows:
1. Title: Represents the Title of the Food Dish.
2. Ingredients: Contains the ingredients as they were scraped from the website.
3. Instructions: Has the recipe instructions to be followed to recreate the dish.

In [13]:
#explore the recipeslmp.csv file
df2 = pd.read_csv("Data\RecipesImp.csv")
display(df2.head())

#display the shape
print(df2.shape)

,title,index,page,about,ingridients,preparation,nutrition per 100g of recipe,energy(kcal),fat(g),carbohydrates(g),proteins(g),fibre(g),vitamin A(mcg),iron(mg),zinc(mg),F_factor_est
0,Kaimati(Fried Dumplings),15003,24,Kaimatis get their unique flavour from the sty...,"wheat flour, refined\r\nwater, vanilla essenc...",Put yeast in a small container.\r\n Add 50ml ...,"Energy 1,795 kJ/ 429 kcal | Fat 21.8 g | Carbo...",429.0,21.8,52.8,4.6,1.6,30,2.1,0.45,0.4
1,Mahamri\r\n(Swahili Doughnut),15004,26,This is a typical traditional recipe among the...,"wheat flour,\r\ncoconut milk\r\nwhite sugar\r\...","Break the coconut shell, drain the water and...","Energy 1,728 kJ/ 413 kcal | Fat 22.1 g | Carbo...",413.0,22.1,46.6,6.0,2.1,41,2.8,0.56,0.4
2,"Enriched Mandazi \r\n(East African Doughnuts, ...",15124,28,A popular snack among urban dwellers across th...,self-raising wheat flour\r\neggs\r\nmargarine\...,"? Put flour, salt, sugar and lemon rind into ...","Energy 1,590 kJ/ 379 kcal | Fat 16.1 g | Carbo...",379.0,16.1,49.9,7.6,2.2,90,3.3,0.66,0.4
3,"Basic Mandazi \r\n(East African Doughnuts, Basic)",15125,30,You will find this recipe in any home across K...,all-purpose wheat flour\r\nbaking powder\r\nsu...,"? Put the wheat flour into a bowl, add baking...","Energy 1,430kJ/ 340 kcal | Fat 12.9 g | Carboh...",340.0,12.9,48.7,6.4,2.1,48,3.5,0.52,0.4
4,Meat Samosa\r\n(Sambusa ya Nyama),15025,32,Nothing more delicious like the Kenyan meaty s...,"minced beef\r\ncoriander, fresh\r\nleek\r\ngar...",? Put the meat in a pan over a fire. Stir con...,"Energy 1,854 kJ/ 443 kcal | Fat 22.2 g | Carbo...",443.0,22.2,40.5,18.8,3.1,66,11.5,2.99,0.4


(142, 16)


- Since we want only a few columns to recommend the possible recipes, we need to drop some columns.

In [14]:
df2.columns


Index(['title', 'index', 'page', 'about', 'ingridients', 'preparation',
       'nutrition per 100g of recipe', 'energy(kcal)', 'fat(g)',
       'carbohydrates(g)', 'proteins(g)', 'fibre(g)', 'vitamin A(mcg)',
       'iron(mg)', 'zinc(mg)', 'F_factor_est'],
      dtype='object')

### Data Cleaning and Sanity check up

1. Check for duplicates and dropping them
2. checking for missing values, drop them or fill them
3. dropping irrelevant columns

In [15]:
columns_to_keep = ['title','ingridients','preparation']

df2 = df2[columns_to_keep]
df2.head()

,title,ingridients,preparation
0,Kaimati(Fried Dumplings),"wheat flour, refined\r\nwater, vanilla essenc...",Put yeast in a small container.\r\n Add 50ml ...
1,Mahamri\r\n(Swahili Doughnut),"wheat flour,\r\ncoconut milk\r\nwhite sugar\r\...","Break the coconut shell, drain the water and..."
2,"Enriched Mandazi \r\n(East African Doughnuts, ...",self-raising wheat flour\r\neggs\r\nmargarine\...,"? Put flour, salt, sugar and lemon rind into ..."
3,"Basic Mandazi \r\n(East African Doughnuts, Basic)",all-purpose wheat flour\r\nbaking powder\r\nsu...,"? Put the wheat flour into a bowl, add baking..."
4,Meat Samosa\r\n(Sambusa ya Nyama),"minced beef\r\ncoriander, fresh\r\nleek\r\ngar...",? Put the meat in a pan over a fire. Stir con...


In [16]:
#clean the column names
#change the ingridient column name to ingredients
df2.rename(columns={'ingridients':'ingredients','preparation':'instructions'}, inplace=True)

#capitalize the column names
df2.columns = df2.columns.str.capitalize()

# Function to process the Ingredients column
def process_ingredients(ingredients):
    # Remove unwanted characters and split by commas
    return [ingredient.strip() for ingredient in ingredients.replace('[','').replace(']','').replace("'", "").replace('\n', ',').split(',')]

# Apply the function to each DataFrame
df['Ingredients'] = df['Ingredients'].apply(lambda x: [ingredient.strip() for ingredient in x.split(',')])
df2['Ingredients'] = df2['Ingredients'].apply(process_ingredients)




In [17]:
# Check the processed DataFrames
df[['Title', 'Ingredients']]


,Title,Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[['1 (3½–4-lb.) whole chicken', '2¾ tsp. koshe..."
1,Crispy Salt and Pepper Potatoes,"[['2 large egg whites', '1 pound new potatoes ..."
2,Thanksgiving Mac and Cheese,"[['1 cup evaporated milk', '1 cup whole milk',..."
3,Italian Sausage and Bread Stuffing,"[['1 (¾- to 1-pound) round Italian loaf, cut i..."
4,Newton's Law,"[['1 teaspoon dark brown sugar', '1 teaspoon h..."
...,...,...
13496,Brownie Pudding Cake,"[['1 cup all-purpose flour', '2/3 cup unsweete..."
13497,Israeli Couscous with Roasted Butternut Squash...,"[['1 preserved lemon', '1 1/2 pound butternut ..."
13498,Rice with Soy-Glazed Bonito Flakes and Sesame ...,[['Leftover katsuo bushi (dried bonito flakes)...
13499,Spanakopita,[['1 stick (1/2 cup) plus 1 tablespoon unsalte...


In [18]:
df2[['Title', 'Ingredients']]

,Title,Ingredients
0,Kaimati(Fried Dumplings),"[wheat flour, refined, water, vanilla essence,..."
1,Mahamri\r\n(Swahili Doughnut),"[wheat flour, , coconut milk, white sugar, dry..."
2,"Enriched Mandazi \r\n(East African Doughnuts, ...","[self-raising wheat flour, eggs, margarine, Ri..."
3,"Basic Mandazi \r\n(East African Doughnuts, Basic)","[all-purpose wheat flour, baking powder, sugar..."
4,Meat Samosa\r\n(Sambusa ya Nyama),"[minced beef, coriander, fresh, leek, garlic, ..."
...,...,...
137,Bhature\r\n (Fried Indian Bread),"[wheat flour, salt, sugar, ghee, cooking oil, ..."
138,Vimumunya vya \r\nSukari\r\n (Sweetened Pumpki...,"[pumpkin, cardamon, sugar, coconut milk, water]"
139,Siro\r\n (Semolina & Nuts),"[semolina flour, cow ghee, cow milk, sugar, pi..."
140,Chaas\r\n(Diluted Yoghurt),"[natural yoghurt, water, salt, ]"


- Chek for missing values

In [19]:
print(f'number of missing values: {df2.isnull().sum().sum()}')

number of missing values: 0


### Merging our datasets
- Since both dataframes have no missing values and duplicates we can merge them now. The new dataset is knowns as Combined_df

In [20]:
#merge the two dataframes
combined_df = pd.concat([df,df2])
#check the shapes of the three dfs
print(f'Dataframe 1 has a shape of: {df.shape}')
print(f'Dataframe 2 has a shape of: {df2.shape}')
print(f'Combined dataframe has a shape of: {combined_df.shape}')

#reset the index
combined_df = combined_df.reset_index(drop=True)



Dataframe 1 has a shape of: (13493, 3)
Dataframe 2 has a shape of: (142, 3)
Combined dataframe has a shape of: (13635, 3)


### Clean the Combined DataFrame

1. Remove newlines and extra spaces
2. Convert string ingredients to lists
3. Clean the Instructions Column

In [21]:
combined_df.head()

,Title,Ingredients,Instructions
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[['1 (3½–4-lb.) whole chicken', '2¾ tsp. koshe...","Pat chicken dry with paper towels, season all ..."
1,Crispy Salt and Pepper Potatoes,"[['2 large egg whites', '1 pound new potatoes ...",Preheat oven to 400°F and line a rimmed baking...
2,Thanksgiving Mac and Cheese,"[['1 cup evaporated milk', '1 cup whole milk',...",Place a rack in middle of oven; preheat to 400...
3,Italian Sausage and Bread Stuffing,"[['1 (¾- to 1-pound) round Italian loaf, cut i...",Preheat oven to 350°F with rack in middle. Gen...
4,Newton's Law,"[['1 teaspoon dark brown sugar', '1 teaspoon h...",Stir together brown sugar and hot water in a c...


In [22]:
combined_df[-10:]

,Title,Ingredients,Instructions
13625,Vinolo\r\n(Banana and Maize Flour Ugali),"[banana green, maize flour, water]",Preparation 5 minutes | Cooking 40 minutes | \...
13626,Finger Millet \r\nFlour Ugali,"[finger millet, water]",Preparation time 5 minutes | Cooking time 15 m...
13627,White Chapati,"[wheat flour, water, sugar, salt, cooking oil]",Preparation 30 minutes | Cooking 30 minutes | ...
13628,Brown Chapati,"[wheat flour, water, sugar, , salt, cooking oil]",Preparation 30 minutes | Cooking 30 minutes | ...
13629,Roti \r\n(Indian Chapati),"[wheat flour, salt, water, cooking oil, cow ghee]",Preparation 3 hours | Cooking 21 minutes | Ser...
13630,Bhature\r\n (Fried Indian Bread),"[wheat flour, salt, sugar, ghee, cooking oil, ...",Preparation 1 hour 15 minutes | Cooking 30 min...
13631,Vimumunya vya \r\nSukari\r\n (Sweetened Pumpki...,"[pumpkin, cardamon, sugar, coconut milk, water]",Preparation 5 minutes | Cooking 45 minutes | \...
13632,Siro\r\n (Semolina & Nuts),"[semolina flour, cow ghee, cow milk, sugar, pi...",Preparation 15 minutes | Cooking 30 minutes | ...
13633,Chaas\r\n(Diluted Yoghurt),"[natural yoghurt, water, salt, ]",Preparation 5 minutes | Serves 2\r\n?Add natur...
13634,Groundnut Sauce,"[groundnut, salt, sour milk, water]",Preparation 5 minutes | Cooking 1 hour 40 minu...


In [23]:
#check for missing values
print(combined_df.isnull().sum().sort_values(ascending=False))


Title           0
Ingredients     0
Instructions    0
dtype: int64


- Clean the columns of the combined_df

In [24]:
#write a function to clean the columns
"""
This function should:
1. Clean the Title Column: Remove newlines and extra spaces
2. Clean the Ingredients Column: Convert string ingredients to lists: If they are not already in a list format, convert them and clean any extra spaces.
                                Ensure there are no empty strings or duplicates within each list of ingredients.
3. Clean the Instructions Column: Similar to the Title, ensure that the instructions are clean and properly formatted.
"""

def clean_combined_df(df):
    #title column
    # Remove newlines and extra spaces
    df['Title'] = df['Title'].str.replace('\n', '').str.strip()
    #Ingredients column
    # Remove empty ingredients from each list of ingredients
    df['Ingredients'] = df['Ingredients'].apply(lambda x: [ingredient.strip() for ingredient in x if ingredient.strip() != ''])

    # If there are any duplicate ingredients in each list, remove them
    df['Ingredients'] = df['Ingredients'].apply(lambda x: list(set(x)))

    # Clean the Instructions column
    df['Instructions'] = df['Instructions'].str.replace('\n', '') \
                                       .str.replace('?', '.') \
                                       .str.replace('|', ',') \
                                       .str.replace('\r', '') \
                                       .str.strip()
    #remove   special characters like \x02 from words like  gradu\x02ally
    df['Instructions'] = df['Instructions'].apply(lambda x: re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', x))#remove special characters
    #if a sentence has more than one fullstop, replace it with one
    df['Instructions'] = df['Instructions'].apply(lambda x: re.sub(r'\.{2,}', '.', x))
    return df

# Use the function on your DataFrame
combined_cleaned = clean_combined_df(combined_df)
combined_cleaned

,Title,Ingredients,Instructions
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['freshly ground pepper'], melted, '2 tsp. whi...","Pat chicken dry with paper towels, season all ..."
1,Crispy Salt and Pepper Potatoes,"['¾ teaspoon finely ground black pepper', '1 t...",Preheat oven to 400F and line a rimmed baking ...
2,Thanksgiving Mac and Cheese,"['4 oz. full-fat cream cheese', '½ tsp. freshl...",Place a rack in middle of oven; preheat to 400...
3,Italian Sausage and Bread Stuffing,"[cut into pieces', casings removed, divided', ...",Preheat oven to 350F with rack in middle. Gene...
4,Newton's Law,['2 teaspoons apple butter (storebought or hom...,Stir together brown sugar and hot water in a c...
...,...,...,...
13630,Bhature\r (Fried Indian Bread),"[water, sugar, cooking oil, wheat flour, fenug...","Preparation 1 hour 15 minutes , Cooking 30 min..."
13631,Vimumunya vya \rSukari\r (Sweetened Pumpkin & ...,"[water, sugar, pumpkin, cardamon, coconut milk]","Preparation 5 minutes , Cooking 45 minutes , S..."
13632,Siro\r (Semolina & Nuts),"[almond, sugar, semolina flour, pistachio nut,...","Preparation 15 minutes , Cooking 30 minutes , ..."
13633,Chaas\r(Diluted Yoghurt),"[natural yoghurt, water, salt]","Preparation 5 minutes , Serves 2.Add natural y..."


In [25]:
combined_cleaned['Instructions'][13634]

'Preparation 5 minutes , Cooking 1 hour 40 minutes , Serves 4.Place a saucepan over fire and let it preheat.Add the groundnuts, salt and 1/2 a cup of water oras desired.Cook until the water evaporates as you stir gradually. When ready, the nuts produce a pop sound.Once they pop, turn down the heat and continue stirring until the groundnuts are dry (about 13minutes).Remove from heat and allow it to cool down.Using a blender, blend the nuts into a paste. Apestle and mortar can be used in the absence of ablender.Put the groundnut paste into a bowl, add sour milkand stir into thick paste. Water or fresh milk can beused in place of the sour milk.Once ready, put another pan on the heat, add thepeanut paste and stir.Stir until it is smooth but not too thick.Serve hot with green leafy vegetables of yourchoice, fish, sweet potatoes, green bananas, ugali,etc'

## Modelling

### Function to get Recipes based on Ingredients
1. Applying tokenization in our ingredient column
2. Defining Function for Ingredients and Title, that is the system is set to work and recommend based on the input, the user can input either title or the ingredients to get the instruction and title or ingredients which will depend with his input
3. Feature Engineering
4. Intiating TF-IDF Vectorizer to  create vectors based on both the recipe title and ingredients.

- Let's see if we can tokenize our Ingredients and see if the system recommends recipes based on the ingredients

In [26]:

def clean_ingredients(ingredients_list):
    # Remove any extra single quotes and fix formatting for each ingredient
    cleaned_list = [re.sub(r"['\"]", "", ingredient) for ingredient in ingredients_list]  # Remove quotes
    cleaned_list = [re.sub(r'\s+', ' ', ingredient) for ingredient in cleaned_list]  # Normalize spaces
    return cleaned_list

# Apply the cleaning function
combined_df['Ingredients'] = combined_df['Ingredients'].apply(clean_ingredients)


# Function to tokenize and normalize ingredients
def tokenize_and_normalize(ingredients_list):
    tokens = []
    for ingredient in ingredients_list:
        # Split ingredient string by commas and strip whitespace
        split_ingredients = [i.strip().lower() for i in ingredient.split(',')]
        
        # Further clean each token: remove unwanted characters
        split_ingredients = [re.sub(r'[^\w\s]', '', i) for i in split_ingredients]  # Remove punctuation
        split_ingredients = [re.sub(r'\s+', ' ', i) for i in split_ingredients]  # Normalize whitespace
        
        # Extend the tokens list with cleaned ingredients
        tokens.extend(split_ingredients)
    
    return tokens


# Apply the function to the Ingredients column
combined_df['Ingredients'] = combined_df['Ingredients'].apply(tokenize_and_normalize)


In [27]:
def recommend_recipes(input_ingredients, combined_df):
    # Normalize user input by stripping whitespace and converting to lower case
    input_ingredients = [ingredient.strip().lower() for ingredient in input_ingredients.split(',')]
    
    # Find matching recipes
    matched_recipes = combined_df[combined_df['Ingredients'].apply(lambda x: any(ingredient in x for ingredient in input_ingredients))]
    
    # Check if any recipes were found
    if matched_recipes.empty:
        return "No recipe found. Try again."
    
    return matched_recipes[['Title','Ingredients', 'Instructions']]


- When the user inputs an ingredient:

In [28]:
# Example user input ingredients
user_input = "thyme"

# Get recommendations
recommended_recipes = recommend_recipes(user_input, combined_df)

# Display recommendations
recommended_recipes


,Title,Ingredients,Instructions
273,Braised Celery With Lentils and Garlic,"[1 bunch celery, freshly ground black pepper, ...",Heat cup oil in a large high-sided skillet ov...
1479,Kale or Chard Pie,"[12 teaspoon baking powder, chervil, sliced, 2...",Heat the oven to 375F. Melt the butter in a la...
1586,Make-Ahead Gravy,"[1 celery stalk, 1 tablespoon vegetable oil, f...",Heat oil in a large saucepan over medium-high....
1897,3-Ingredient Garlic-Herb Grilled Chicken Wings,[2 pounds chicken wings flats and drumettes at...,Pat wings very dry with paper towels. Whisk ga...
2766,Herby Ricotta-Poblano Tacos,"[2 fresh poblano chiles, epazote, 3 or 4 radis...","On an ungreased griddle or small, heavy skille..."
5510,Grilled Fish,"[2 pounds fish steaks, thyme, or dry white win...",Combine all the ingredients except fish and le...
5988,Gazpacho,"[1 red bell pepper, 14 cup red wine vinegar, k...","1. In a bowl, reserve 2 tablespoons each of th..."
6253,Duck Breast & Zucchini Tournedos,"[thyme, 1 34 oz dried cep mushrooms champagne ...","For the mushroom powder, process the dried cep..."
6851,Chicken & Rice Soup,"[peeled and diced, longgrain white rice 13 cup...","Combine the broth, rice, and 2 cups (16 fl oz/..."
7203,Grilled Chicken with Almond and Garlic Sauce,"[3 tablespoons dry white wine, such as parsley...","1. For the sauce, soak the beans in a generous..."


- The results are titles, ingredients and instructions all with the name thyme in it.

- When the user enters a title:

In [29]:
# Example user input ingredients
user_input = "Gazpacho"

# Get recommendations
recommended_recipes = recommend_recipes(user_input, combined_df)

# Display recommendations
recommended_recipes


'No recipe found. Try again.'

In [30]:
#check if there is a recipe with the title 'Gazpacho'
combined_df[combined_df['Title'].str.contains('Gazpacho', case=False)]

,Title,Ingredients,Instructions
133,Speedy Summer Gazpacho,"[1 celery stalk, 2 garlic cloves, grated, 6 me...",Put all the ingredients in a food processor an...
1771,Sippin’ Green Gazpacho,"[2 garlic cloves, 2 pounds english hothouse cu...","Pure cucumbers, garlic, and 1/2 cup water in a..."
3894,Gochujang Gazpacho,"[2 garlic cloves, 2 cups diced ripe tomato, 2 ...","Pulse 1 1/2 cups tomato, 1/4 cup cucumber, ora..."
4089,Wee Gazpacho,"[1 cucumber, 1 teaspoon lemon juice, 2 celery ...",Place all the ingredients in a food processor ...
4476,Watermelon Gazpacho with Feta Crema,"[rind removed, kosher salt, 1 pound seedless w...","Pure watermelon, tomato, cucumber, jalapeo, oi..."
5297,Creamy Green Gazpacho,"[1 small celery stalk optional, freshly ground...","Reserve one-quarter of the tomato, two cucumbe..."
5988,Gazpacho,"[1 red bell pepper, 14 cup red wine vinegar, k...","1. In a bowl, reserve 2 tablespoons each of th..."
6163,Mixed Berry Gazpacho with Basil,"[torn into pieces, 1 teaspoon fresh lemon juic...",Combine first 6 ingredients and basil sprig in...
6260,Stone Fruit Gazpacho with Scallops,[2 cups cubed yellow or red seedless watermelo...,"Combine the peaches, plums, watermelon, garlic..."
6564,Fiery Grilled Shrimp with Honeydew Gazpacho,"[and chopped, plus more to taste, coarsely cho...",Cut away the honeydew rind and the dark green ...


- They get the response "No recipe found.Try again" but if you check among the foods, there is a recipe for Gazpacho.

- In the above function we see that when a user enters an ingredient it results in sample recipes the user can cook. However when the user tries to enter a title of a food they want, it returns the response "No recipe found. Try Again."
- If you search the title of the food in the list of food, it's there. So the function isn't really working accordingly.
- Thus we can try Feature Engineering. Create a new column with combined title and ingredients, Vectorize, create cosine similarities and see how well it performs compared to our function above.

### Feature Engineering

- Create the column containing combined title and ingredient.

In [31]:
# Combining title and ingredients
"""
This code is meant to combine both the ingredients and the title into a 
single string for each recipe this will allow to system to search both 
the ingredients and the title since ther user can input any of them.
"""
combined_df['Title_Ingredients'] = combined_df['Title'] + ' ' + combined_df['Ingredients'].apply(lambda x: ' '.join(x))
combined_df.head()

,Title,Ingredients,Instructions,Title_Ingredients
0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"[freshly ground pepper, melted, 2 tsp white mi...","Pat chicken dry with paper towels, season all ...",Miso-Butter Roast Chicken With Acorn Squash Pa...
1,Crispy Salt and Pepper Potatoes,"[¾ teaspoon finely ground black pepper, 1 teas...",Preheat oven to 400F and line a rimmed baking ...,Crispy Salt and Pepper Potatoes ¾ teaspoon fin...
2,Thanksgiving Mac and Cheese,"[4 oz fullfat cream cheese, ½ tsp freshly grou...",Place a rack in middle of oven; preheat to 400...,Thanksgiving Mac and Cheese 4 oz fullfat cream...
3,Italian Sausage and Bread Stuffing,"[cut into pieces, casings removed, divided, 2 ...",Preheat oven to 350F with rack in middle. Gene...,Italian Sausage and Bread Stuffing cut into pi...
4,Newton's Law,[2 teaspoons apple butter storebought or homem...,Stir together brown sugar and hot water in a c...,Newton's Law 2 teaspoons apple butter storebou...


In [32]:
# preview combined column of title together with ingredients
combined_df['Title_Ingredients'][0]

'Miso-Butter Roast Chicken With Acorn Squash Panzanella freshly ground pepper melted 2 tsp white miso 2 small acorn squash about 3 lb total freshly ground black pepper 6 tbsp unsalted butter cored ½ small red onion kosher salt 2 tbsp finely chopped sage pinch of crushed red pepper flakes 1 tbsp finely chopped rosemary 2¾ tsp kosher salt 1 tbsp white miso 2 tbsp unsalted butter 1 3½4lb whole chicken divided room temperature ¼ cup allpurpose flour 2 medium apples such as gala or pink lady about 14 oz total cut into 1 pieces 3 tbsp apple cider vinegar plus more 2 cups unsalted chicken broth torn into 1 pieces about 2½ cups ¼ cup dry white wine ¼ tsp ground allspice thinly sliced 2 tbsp extravirgin olive oil plus 3 tbsp room temperature ⅓ loaf goodquality sturdy white bread'

### TF-IDF Vectorizer
-The TF-IDF vectorizer will then create vectors based on both the recipe title and ingredients. This will give you a more comprehensive representation of the recipe.

In [33]:
# intiliazing TFIDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
# fit the vectorizer
tfidf_matrix = tfidf_vectorizer.fit_transform(combined_df['Title_Ingredients'])


- Check the shape of the matrix and see the features to know how the vectorization has taken place.

In [34]:
# Check the shape of the TF-IDF matrix (rows x features)
print(f'Shape of matrix:{tfidf_matrix.shape}')

# Get the feature names (terms)
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
print(f'Feature Names:{tfidf_feature_names[:10]}')  # Show first 10 features (terms)

Shape of matrix:(13635, 11974)
Feature Names:['00' '000mg' '011' '018' '018oz' '02' '025' '028' '035' '037']


- In the above we can see how the shape of the matrix is 13,635 by 13,690 after vectorization and sample feature names.

- Let's get the cosine Similarities

In [35]:
#cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

#print the cosine similarity matrix shape
print(f'Cosine Similarity Matrix Shape: {cosine_sim.shape}')


Cosine Similarity Matrix Shape: (13635, 13635)


- Let's see how well our system recommends recipes based on the cosine similarities.
- Let's take a look at the first recipe and the top 5 recipes with highest cosine similarities with it.

In [36]:
#example getting top 5 most similar recipes with the first recipe
similar_recipes = cosine_sim[0].argsort()[-6:-1] #


In [37]:
#display the first recipe
combined_df.iloc[0][['Title', 'Ingredients', 'Instructions']]

Title           Miso-Butter Roast Chicken With Acorn Squash Pa...
Ingredients     [freshly ground pepper, melted, 2 tsp white mi...
Instructions    Pat chicken dry with paper towels, season all ...
Name: 0, dtype: object

In [38]:
# Display the similar recipes
combined_df.iloc[similar_recipes][['Title', 'Ingredients', 'Instructions']][::-1]

,Title,Ingredients,Instructions
590,Grilled Whole Cauliflower with Miso Mayo,"[12 tsp or more kosher salt, 14 cup vinegarbas...",Prepare a grill for medium-high heat. Sprinkle...
129,Grilled Potatoes With Red Miso Butter,"[1 garlic clove, 1½ lb baby yukon gold potatoe...",Prepare a grill for medium-high heat. Bring po...
331,Risotto With Mushrooms and Thyme,"[cut into pieces, 2 tbsp unsalted butter, ¼ cu...",Combine 1 Tbsp. salt and 10 cups water in a me...
4237,Bacon-Cheddar Muffins,"[34 cup cornmeal, coarsely chopped, 3 eggs, di...",Preheat oven to 400 with rack near top. Line t...
26,Miso-Squash Ramen,"[kosher salt, 1 1 piece ginger, freshly ground...","Preheat oven to 400F. Whisk oil, miso, maple s..."


In [39]:
#display the similarities of the first recipe with the top 5 similar recipes
cosine_sim[0][similar_recipes][::-1].round(4)

array([0.4927, 0.4799, 0.4646, 0.4638, 0.4568])

- In the above, there have been 5 top recipe recommendations  based on their cosine similarities with the first recipe. The first recipe having the highest similarity of 0.4927

- Clean the title.

In [40]:
# clean title column
def clean_input(title):
    # Normalize titles by removing punctuation, converting to lowercase, and stripping extra spaces
    title = re.sub(r'[^\w\s]', '', title)
    title = re.sub(r'\s+', ' ', title).strip().lower()
    return title
combined_df['Title'] = combined_df['Title'].apply(clean_input)

- Create a function to recommend the recipes.

In [41]:
#function to recommend recipes
def recommend_recipe(user_input, combined_df, tfidf_vectorizer, tfidf_matrix, n=10):
    # Clean the user input
    user_input_cleaned = clean_input(user_input)
    
    # Check if the user input matches a specific title
    matching_title = combined_df[combined_df['Title'].str.lower().str.contains(user_input_cleaned)]
    
    #check if there is a match by title
    if not matching_title.empty:
        return matching_title[['Title', 'Ingredients', 'Instructions']]
    
    # If no exact title match is found, proceed with TF-IDF based recommendation
    # Transform the user input
    user_input_transformed = tfidf_vectorizer.transform([user_input_cleaned])
    
    # Compute the cosine similarity between the user input and the recipes
    cosine_sim = cosine_similarity(user_input_transformed, tfidf_matrix)
    
    # Get the index of the top n most similar recipes
    similar_recipes = cosine_sim[0].argsort()[-n:][::-1]
    
    # Return the top n most similar recipes
    return combined_df.iloc[similar_recipes][['Title', 'Ingredients', 'Instructions']]
    

In [42]:
#Example usage
user_input = "eggs,tomatoes,onions"
recommendations = recommend_recipe(user_input, combined_df, tfidf_vectorizer, tfidf_matrix)

# View recommendations
recommendations

,Title,Ingredients,Instructions
13634,groundnut sauce,"[water, salt, sour milk, groundnut]","Preparation 5 minutes , Cooking 1 hour 40 minu..."
4552,charred and raw corn with chile and cheese,"[with seeds, 14 cup cilantro leaves with tende...",Prepare grill for medium heat. Cut kernels fro...
4550,grilled rib eye with paprika vinaigrette,"[14 cup chopped fresh flatleaf parsley, 2 32ou...",Heat oil and paprika in a small saucepan over ...
4549,grilled zucchini and leeks with walnuts and herbs,[12 cup lightly packed fresh flatleaf parsley ...,Prepare grill for medium-high heat. Toast waln...
4548,asparagus with bacon and hardboiled eggs,"[trimmed, 1 tablespoon pure maple syrup, 1 tab...",Cook bacon in a large skillet over medium heat...
4547,mint and cuminspiced lamb chops,"[1 teaspoon raselhanout or garam masala, 1 cup...","Pulse onions, cilantro, parsley, mint, cumin, ..."
4546,moroccan chicken brochettes,"[labneh lebanese strained yogurt, kosher salt,...",Place garlic in a mortar; season with salt and...
4545,grilled sesame squid,"[freshly ground pepper, kosher salt, 2 tablesp...",Prepare grill for high heat; oil grate with ve...
4544,plum tarts with honey and black pepper,"[apricots, 1 sheet frozen puff pastry one 14ou...","Preheat oven to 425. Cut pastry into six 4"" sq..."
4543,pickled carrots with tarragon,"[peeled, trimmed, 1 tablespoon black peppercor...","Pack carrots, shallot, garlic, chiles, tarrago..."


In [43]:
#Example usage
user_input = "githeri"
recommendations = recommend_recipe(user_input, combined_df, tfidf_vectorizer, tfidf_matrix)

# View recommendations
recommendations

,Title,Ingredients,Instructions
13545,githeri fresh beans and maize,"[water, cooking fat, spring onion, bean pods, ...","Preparation 2 hours , Cooking 20 minutes , Ser..."
13546,githeri stewed maize beans,"[water, onion, cooking oil, maize and beans, s...","Preparation 10 minutes , Cooking 15 minutes , ..."
13547,githeri sauted fresh maize beans,"[water, onion, cooking fat, kidney beans, gree...","Preparation 5 hours 30 minutes , Cooking 10 mi..."


- Based on the above, the system works in a way that when a user enters a specific title of a food like "mandazi" it returns all instances where the food title is seen otherwise, if the user enters ingredients, it converts the user input to a vector then performs the cosine similarity. Afterwards, it returns recipes of the top foods with highest cosine similarities.

- This system works accordingly in recommending the recipes to cook.


## Deployment

### Saving Code
- In order for a user to get recommended recipes, we need to save some code such as the recommend recipe function, combined_df, the tf_idf vectorizer as
.py and pickle files in order for a user to get feedback whenever they input data.
- So the TF-IDF Vectorizer and the combined dataframe will be saved as pickle files while the combined_df saved as .py file to make it easy to adjust here and there.

In [44]:
# Saving the model - CORRECTED VERSION
from joblib import dump
import os

# # Create Model directory if it doesn't exist
# os.makedirs('Model', exist_ok=True)

# Save the tfidf vectorizer (using .joblib extension recommended)
dump(tfidf_vectorizer, 'Model/tfidf_vectorizer.joblib')

# Save the tfidf matrix
dump(tfidf_matrix, 'Model/tfidf_matrix.joblib')

# # Verify the files were saved
# print("Models saved successfully!")
# print(f"Vectorizer size: {os.path.getsize('Model/tfidf_vectorizer.joblib')} bytes")
# print(f"Matrix size: {os.path.getsize('Model/tfidf_matrix.joblib')} bytes")

['Model/tfidf_matrix.joblib']

In [45]:
# #saving the model
# import joblib

# #save the tfidf vectorizer
# joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.pkl')

# #save the tfidf matrix
# joblib.dump(tfidf_matrix, 'tfidf_matrix.pkl')

# #save the combined dataframe
# combined_df.to_csv('combined_df.csv', index=False)
